In [1]:
import re
from itertools import groupby

def parse_input():
    with open("input.txt") as f:
        input_txt = f.read()
    nodes_disk_info_txt = input_txt.splitlines()[2:]
    nodes_disk_info = dict()
    for node_disk_info_txt in nodes_disk_info_txt:
        parsed, = re.findall(r"/dev/grid/node-x(\d+)-y(\d+)\s+(\d+)T\s+(\d+)T\s+(\d+)T\s+(\d+)%", node_disk_info_txt)
        (x, y, size, used, avail, pct_used) = [int(p) for p in parsed]
        nodes_disk_info[(x, y)] = (size, used, avail, pct_used)
    return nodes_disk_info


nodes_disk_info = parse_input()


In [2]:
max_x = max(x for x, y in nodes_disk_info.keys())
max_y = max(y for x, y in nodes_disk_info.keys())
target_pos = (max_x, 0)
target_data = nodes_disk_info[target_pos]
print("Target pos", target_pos)
print(f"Max x {max_x}, Max y {max_y}")

Target pos (36, 0)
Max x 36, Max y 24


## PART 1

In [3]:
viable_pairs = []
for n1, (n1_size, n1_used, n1_avail, n1_pct_used) in nodes_disk_info.items():
    for n2, (n2_size, n2_used, n2_avail, n2_pct_used) in nodes_disk_info.items():
        if n1 != n2 and 0 < n1_used < n2_avail:
            viable_pairs.append((n1, n2))

In [4]:
print(f"Part 1 solution: {len(viable_pairs)}")

Part 1 solution: 888


## PART 2

In [5]:
used_sizes = [(used, pos) for (pos, (_, used, _, _)) in nodes_disk_info.items()]
avail_sizes = [(avail, pos) for (pos, (_, _, avail, _)) in nodes_disk_info.items()]
tot_sizes = [(size, pos) for (pos, (size, _, _, _)) in nodes_disk_info.items()]

used_sizes = sorted(used_sizes, key=lambda x: x[0])
avail_sizes = sorted(avail_sizes, key=lambda x: x[0])
tot_sizes = sorted(tot_sizes, key=lambda x: x[0])

In [6]:
# only (19, 6) has enough space available for any move => the max possible move is 93T
print("Max avail space", avail_sizes[::-1][:5])
print("Min used space", used_sizes[:5])
max_possible_move = avail_sizes[::-1][0][0]
print("Max possible move", max_possible_move)
start_pos = avail_sizes[::-1][0][1]
print("Starting empty pos", start_pos)

Max avail space [(93, (19, 6)), (30, (32, 8)), (30, (27, 9)), (30, (21, 17)), (30, (20, 21))]
Min used space [(0, (19, 6)), (64, (0, 2)), (64, (0, 3)), (64, (0, 10)), (64, (0, 13))]
Max possible move 93
Starting empty pos (19, 6)


In [7]:
# In row 3 only 1 disk that can be freed: (0, 3)
impossible_moves = sorted([us for us in used_sizes[::-1] if us[0] > max_possible_move], key=lambda e: (e[1][1], e[1][0]))
print("Impossible moves", impossible_moves)

possible_passages = []
for y, imp_moves_group in groupby(impossible_moves, key=lambda e: e[1][1]):
    possible_xs = set(range(max_x+1)) - set([e[1][0] for e in imp_moves_group])
    possible_passages.extend([(x, y) for x in possible_xs])
    print(f"Possible x passages for y={y}", possible_xs)

passage_pos, = possible_passages
print("Only possible passage", passage_pos)

Impossible moves [(493, (1, 3)), (490, (2, 3)), (490, (3, 3)), (494, (4, 3)), (492, (5, 3)), (497, (6, 3)), (496, (7, 3)), (497, (8, 3)), (495, (9, 3)), (494, (10, 3)), (495, (11, 3)), (492, (12, 3)), (499, (13, 3)), (495, (14, 3)), (497, (15, 3)), (492, (16, 3)), (495, (17, 3)), (496, (18, 3)), (497, (19, 3)), (494, (20, 3)), (491, (21, 3)), (492, (22, 3)), (494, (23, 3)), (493, (24, 3)), (499, (25, 3)), (491, (26, 3)), (495, (27, 3)), (491, (28, 3)), (491, (29, 3)), (491, (30, 3)), (499, (31, 3)), (498, (32, 3)), (492, (33, 3)), (499, (34, 3)), (493, (35, 3)), (499, (36, 3))]
Possible x passages for y=3 {0}
Only possible passage (0, 3)


In [8]:
# If freed, there is no disk that cannot contain target data
target_used = target_data[1]
print("Target disk used", target_used)
impossible_passages = [ts for ts in tot_sizes if ts[0] < target_used]
print("Disks with tot size lower than target data", impossible_passages)

Target disk used 67
Disks with tot size lower than target data []


In [9]:
# "Moving" an empty space 1 pos away requires 1 step
# Given the observations above, (19, 6) can be "moved" anywhere, and any space freed can contain target data
# => all pos are equivalent except the barrier at y=3

# First passage
sx, sy = start_pos
px, py = passage_pos
first_passage_dist = abs(sx - px) + abs(sy - py)
print("First passage dist", first_passage_dist)

First passage dist 22


In [10]:
# Move the empty space to the position left to the target
tx, ty = target_pos
target_left_pos = (tx-1, ty)
tlx, tly = target_left_pos
print("Position target left", target_left_pos)
passage_to_left_dist = abs(px - tlx) + abs(py - tly)
print("Passage to left dist", passage_to_left_dist)

Position target left (35, 0)
Passage to left dist 38


In [11]:
# Switch empty and target => add 1 dist
switch_empty_target_dist = 1

# Once the empty is on the right, each move to the left for the target requires 5 steps:
# 4 for the empty to reach the position left of the target + 1 switch
switch_target_empty_dist = 5

remaining_dx = tlx
remaining_dist = remaining_dx * switch_target_empty_dist
print("Remaining dx", remaining_dx)
print("Remaining dist", remaining_dist)

Remaining dx 35
Remaining dist 175


In [12]:
tot_dist = first_passage_dist + passage_to_left_dist + switch_empty_target_dist + remaining_dist

In [13]:
print(f"Part 2 solution: {tot_dist}")

Part 2 solution: 236
